# Core learning algorithms




In [3]:
# import stuffs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from six.moves import urllib
import tensorflow.compat.v2.feature_column as fc


## Get data
The code below is what i found in github:  
https://github.com/tensorflow/datasets/blob/master/tensorflow_datasets/structured/titanic.py

In [2]:
import tensorflow as tf
from tensorflow_datasets.core import lazy_builder_import

Titanic = lazy_builder_import.LazyBuilderImport('titanic')
type(Titanic) # tensorflow_datasets.core.lazy_builder_import.LazyBuilderImport

c:\Users\eziod\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


tensorflow_datasets.core.lazy_builder_import.LazyBuilderImport

This is what the instructor used. The data is stored on the remote:  
https://storage.googleapis.com/tf-datasets/titanic/train.csv  
https://storage.googleapis.com/tf-datasets/titanic/eval.csv  
To avoid calling api each time i run the script, i downloaded the csv.

In [5]:
# load dataset
dftrain = pd.read_csv("./data/titanic/train.csv") # training data
dfeval = pd.read_csv("./data/titanic/eval.csv") # testing data
y_train = dftrain.pop("survived") # "survived" is a column's name
y_eval = dfeval.pop("survived")
# take a look
print(dftrain.head())

      sex   age  n_siblings_spouses  parch     fare  class     deck  \
0    male  22.0                   1      0   7.2500  Third  unknown   
1  female  38.0                   1      0  71.2833  First        C   
2  female  26.0                   0      0   7.9250  Third  unknown   
3  female  35.0                   1      0  53.1000  First        C   
4    male  28.0                   0      0   8.4583  Third  unknown   

   embark_town alone  
0  Southampton     n  
1    Cherbourg     n  
2  Southampton     y  
3  Southampton     n  
4   Queenstown     y  


A statistical analysis. Use pandas' .describe() method.

In [6]:
dftrain.describe()

,age,n_siblings_spouses,parch,fare
count,627.000000,627.000000,627.000000,627.000000
mean,29.631308,0.545455,0.379585,34.385399
std,12.511818,1.151090,0.792999,54.597730
min,0.750000,0.000000,0.000000,0.000000
25%,23.000000,0.000000,0.000000,7.895800
50%,28.000000,0.000000,0.000000,15.045800
75%,35.000000,1.000000,0.000000,31.387500
max,80.000000,8.000000,5.000000,512.329200
